In [1]:
!git clone https://github.com/hoangngoclam298/Siamese-Triplet-Model.git

Cloning into 'Siamese-Triplet-Model'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 89 (delta 30), reused 73 (delta 14), pack-reused 0
Unpacking objects: 100% (89/89), 4.05 MiB | 11.75 MiB/s, done.


In [2]:
%cd /kaggle/working/Siamese-Triplet-Model

/kaggle/working/Siamese-Triplet-Model


In [3]:
from torchvision import datasets, transforms
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import Dataset, DataLoader
import random
import torch
import shutil
import time
from torch.optim import lr_scheduler
import torch.optim as optim
from networks import *
from datasets import *
from losses import *
from metrics import *
# from trainer import *
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
path_root = '/kaggle/input/new-data-face/data_train_2d_new/data_train_2d'
new_path_train = '/kaggle/working/data_train_2d'
new_path_test = '/kaggle/working/data_test_2d'

if os.path.exists(new_path_train):
    shutil.rmtree(new_path_train)
os.makedirs(new_path_train, exist_ok=True)

if os.path.exists(new_path_test):
    shutil.rmtree(new_path_test)
os.makedirs(new_path_test, exist_ok=True)

list_id = os.listdir(path_root)
random.shuffle(list_id)
list_train = ['1038', '1190', '1187', '1041', '2175', '1134', '2059', '1220', '1119', '2198', '2005', '1031', '1008', '1042', '1221', '1080', '2170', '1032', '2029', '2046', '1075', '2017', '1012', '2213', '2023', '2002', '2072', '2197', '1089', '2183', '1027', '1189', '2215', '2200', '2191', '2214', '1186', '2193', '1182', '1002', '2223', '1050', '1188', '1133', '1200', '2075', '1151', '1116', '2006', '2207', '1022', '1208', '1024', '1199', '1088', '2177', '2009', '2185', '1195', '1213', '2051', '2021', '1099', '2052', '2019', '2218', '2205', '2226', '1013', '2173', '1003', '1144', '1175', '1085', '1196', '2209', '1074', '2068', '2188', '2195', '2086', '2016', '1039', '1217', '2011', '2186', '1037', '1181', '1173', '2203', '1066', '1004', '2180', '2008', '1201', '1197', '1192', '2201', '1174', '1184', '2060', '2064', '2181', '2012']
list_test = ['2172', '1040', '2189', '2174', '2184', '1043', '2219', '2179', '2225', '1072', '2210', '2074', '1001', '2001', '1014', '2176', '1177', '2141', '2004', '1067', '2211', '2054', '2178', '1047', '2190', '1025', '1009']

for id in list_train:
    path_id = os.path.join(path_root, id)
    list_id_image = [x for x in os.listdir(path_id) if x.endswith('.png')]
    if len(list_id_image) > 1:
        if len(list_id_image) > 5:
            list_id_image = random.sample(list_id_image, 5)
        os.makedirs(os.path.join(new_path_train, id), exist_ok=True)
        for id_image in list_id_image:
            source_file = os.path.join(path_id, id_image)
            destination_file = os.path.join(new_path_train, id, id_image)
            shutil.copy(source_file, destination_file)

for id in list_test:
    path_id = os.path.join(path_root, id)
    list_id_image = [x for x in os.listdir(path_id) if x.endswith('.png')]
    if len(list_id_image) > 1:
        if len(list_id_image) > 5:
            list_id_image = random.sample(list_id_image, 5)
        os.makedirs(os.path.join(new_path_test, id), exist_ok=True)
        for id_image in list_id_image:
            source_file = os.path.join(path_id, id_image)
            destination_file = os.path.join(new_path_test, id, id_image)
            shutil.copy(source_file, destination_file)
            

            
path_root = '/kaggle/input/new-data-face/data_train_3d_new/data_train_3d'
new_path_train = '/kaggle/working/data_train_3d'
new_path_test = '/kaggle/working/data_test_3d'

if os.path.exists(new_path_train):
    shutil.rmtree(new_path_train)
os.makedirs(new_path_train, exist_ok=True)

if os.path.exists(new_path_test):
    shutil.rmtree(new_path_test)
os.makedirs(new_path_test, exist_ok=True)

list_id = os.listdir(path_root)
random.shuffle(list_id)
list_train = ['1038', '1190', '1187', '1041', '2175', '1134', '2059', '1220', '1119', '2198', '2005', '1031', '1008', '1042', '1221', '1080', '2170', '1032', '2029', '2046', '1075', '2017', '1012', '2213', '2023', '2002', '2072', '2197', '1089', '2183', '1027', '1189', '2215', '2200', '2191', '2214', '1186', '2193', '1182', '1002', '2223', '1050', '1188', '1133', '1200', '2075', '1151', '1116', '2006', '2207', '1022', '1208', '1024', '1199', '1088', '2177', '2009', '2185', '1195', '1213', '2051', '2021', '1099', '2052', '2019', '2218', '2205', '2226', '1013', '2173', '1003', '1144', '1175', '1085', '1196', '2209', '1074', '2068', '2188', '2195', '2086', '2016', '1039', '1217', '2011', '2186', '1037', '1181', '1173', '2203', '1066', '1004', '2180', '2008', '1201', '1197', '1192', '2201', '1174', '1184', '2060', '2064', '2181', '2012']
list_test = ['2172', '1040', '2189', '2174', '2184', '1043', '2219', '2179', '2225', '1072', '2210', '2074', '1001', '2001', '1014', '2176', '1177', '2141', '2004', '1067', '2211', '2054', '2178', '1047', '2190', '1025', '1009']

for id in list_train:
    path_id = os.path.join(path_root, id)
    list_id_image = [x for x in os.listdir(path_id) if x.endswith('.png')]
    if len(list_id_image) > 1:
        if len(list_id_image) > 5:
            list_id_image = random.sample(list_id_image, 5)
        os.makedirs(os.path.join(new_path_train, id), exist_ok=True)
        for id_image in list_id_image:
            source_file = os.path.join(path_id, id_image)
            destination_file = os.path.join(new_path_train, id, id_image)
            shutil.copy(source_file, destination_file)

for id in list_test:
    path_id = os.path.join(path_root, id)
    list_id_image = [x for x in os.listdir(path_id) if x.endswith('.png')]
    if len(list_id_image) > 1:
        if len(list_id_image) > 5:
            list_id_image = random.sample(list_id_image, 5)
        os.makedirs(os.path.join(new_path_test, id), exist_ok=True)
        for id_image in list_id_image:
            source_file = os.path.join(path_id, id_image)
            destination_file = os.path.join(new_path_test, id, id_image)
            shutil.copy(source_file, destination_file)

In [5]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class ConcatDataset(Dataset):
    def __init__(self, data_dir1, data_dir2, transform=None):
        self.data_dir1 = data_dir1
        self.data_dir2 = data_dir2
        self.transform = transform
        self.classes = sorted(os.listdir(os.path.join(data_dir1)))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.img_paths = self._get_img_paths()

    def _get_img_paths(self):
        img_paths = []
        for cls in self.classes:
            cls_dir1 = os.path.join(self.data_dir1, cls)
            cls_dir2 = os.path.join(self.data_dir2, cls)
            for img_name in os.listdir(cls_dir1):
                img_path1 = os.path.join(cls_dir1, img_name)
                img_path2 = os.path.join(cls_dir2, img_name)
                img_paths.append((img_path1, img_path2, self.class_to_idx[cls]))
        return img_paths

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path1, img_path2, label = self.img_paths[idx]
        img1 = Image.open(img_path1).convert("RGB")
        img2 = Image.open(img_path2).convert("RGB")

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, label

In [6]:
cuda = torch.cuda.is_available()

trans_train = transforms.Compose([
#     transforms.RandomRotation(degrees=(-15, 15)),
    RandomResizedCropRect(size=(256, 256)),
    GaussianNoise(),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

trans_val = transforms.Compose([
#     transforms.RandomRotation(degrees=(-15, 15)),
    transforms.Resize((256, 256)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

new_path_train1 = '/kaggle/working/data_train_2d'
new_path_test1 = '/kaggle/working/data_test_2d'
new_path_train2 = '/kaggle/working/data_train_3d'
new_path_test2 = '/kaggle/working/data_test_3d'

train_dataset = TripletDatasetConcat(data_dir1=new_path_train1, data_dir2=new_path_train2, transform=trans_train, train=True)
test_dataset = TripletDatasetConcat(data_dir1=new_path_test1, data_dir2=new_path_test2, transform=trans_val, train=False)

batch_size = 8
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

workers = 0 if os.name == 'nt' else 8

train_dataset_image = ConcatDataset(data_dir1=new_path_train1, data_dir2=new_path_train2, transform=trans_train)
train_loader_image = DataLoader(
    train_dataset_image,
    num_workers=workers,
    batch_size=batch_size,
)

val_dataset_image = ConcatDataset(data_dir1=new_path_test1, data_dir2=new_path_test2, transform=trans_val)
val_loader_image = DataLoader(
    val_dataset_image,
    num_workers=workers,
    batch_size=batch_size,
)

In [7]:
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import torch.nn.functional as F


def fit(train_loader, val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[],
        start_epoch=0, save_best_loss=False, path_save='', num_epoch_roc = -1, train_loader_image=None, val_loader_image=None):
    """
    Loaders, model, loss function and metrics should work together for a given task,
    i.e. The model should be able to process data output of loaders,
    loss function should process target output of loaders and outputs from the model

    Examples: Classification: batch loader, classification model, NLL loss, accuracy metric
    Siamese network: Siamese loader, siamese model, contrastive loss
    Online triplet learning: batch loader, embedding model, online triplet loss
    """
    best_loss = 1000
    best_model = None

    for epoch in range(0, start_epoch):
        scheduler.step()

    for epoch in range(start_epoch, n_epochs):
        scheduler.step()

        # Train stage
        train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)

        message = 'Epoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch + 1, n_epochs, train_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        val_loss, metrics = test_epoch(val_loader, model, loss_fn, cuda, metrics)
        val_loss /= len(val_loader)
        if save_best_loss:
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = model.state_dict().copy()
        message += '\nEpoch: {}/{}. Validation set: Average loss: {:.4f}'.format(epoch + 1, n_epochs,
                                                                                 val_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        print(message)
        if num_epoch_roc != -1:
            if (epoch + 2) % num_epoch_roc == 0:
                roc_auc_dis, roc_auc_cos = check_ROC(train_loader_image, model, cuda)
                print('---Epoch: {}/{}. Train set: Distance ROC: {:.4f}\tCosine similarity ROC: {:.4f}'
                    .format(epoch + 1, n_epochs, roc_auc_dis, roc_auc_cos))
                roc_auc_dis, roc_auc_cos = check_ROC(val_loader_image, model, cuda)
                print('---Epoch: {}/{}. Validation set: Distance ROC: {:.4f}\tCosine similarity ROC: {:.4f}'
                    .format(epoch + 1, n_epochs, roc_auc_dis, roc_auc_cos))
    if save_best_loss:
        torch.save(best_model, path_save)


def train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics):
    for metric in metrics:
        metric.reset()

    model.train()
    losses = []
    total_loss = 0

    for batch_idx, (data, target) in (enumerate(train_loader)):
        target = target if len(target) > 0 else None
        if not type(data) in (tuple, list):
            data = (data,)
        if cuda:
            data = tuple(d.cuda() for d in data)
            if target is not None:
                target = target.cuda()


        optimizer.zero_grad()
        outputs = model(*data)

        if type(outputs) not in (tuple, list):
            outputs = (outputs,)

        loss_inputs = outputs
        if target is not None:
            target = (target,)
            loss_inputs += target

        loss_outputs = loss_fn(*loss_inputs)
        loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
        losses.append(loss.item())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        for metric in metrics:
            metric(outputs, target, loss_outputs)

        # if batch_idx % log_interval == 0:
        #     message = 'Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         batch_idx * len(data[0]), len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader), np.mean(losses))
        #     for metric in metrics:
        #         message += '\t{}: {}'.format(metric.name(), metric.value())

        #     print(message)
        #     losses = []

    total_loss /= (batch_idx + 1)
    return total_loss, metrics


def test_epoch(val_loader, model, loss_fn, cuda, metrics):
    with torch.no_grad():
        for metric in metrics:
            metric.reset()
        model.eval()
        val_loss = 0
        for batch_idx, (data, target) in enumerate(val_loader):
            target = target if len(target) > 0 else None
            if not type(data) in (tuple, list):
                data = (data,)
            if cuda:
                data = tuple(d.cuda() for d in data)
                if target is not None:
                    target = target.cuda()

            outputs = model(*data)

            if type(outputs) not in (tuple, list):
                outputs = (outputs,)
            loss_inputs = outputs
            if target is not None:
                target = (target,)
                loss_inputs += target

            loss_outputs = loss_fn(*loss_inputs)
            loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
            val_loss += loss.item()

            for metric in metrics:
                metric(outputs, target, loss_outputs)

    return val_loss, metrics

def check_ROC(val_loader, model, device):
    with torch.no_grad():
        model.eval()
        outputs_list = []

        for batch in val_loader:
            image1, image2, ids = batch  # Giả sử batch trả về (ảnh, nhãn)
            image1 = image1.cuda()
            image2 = image2.cuda()
            outputs = model.get_embedding(image1, image2)
            
            for i in range(outputs.size(0)):
                outputs_list.append((ids[i].item(), outputs[i].cpu().numpy()))  # Lưu ID và output
        scores_dis = []
        labels_dis = []
        scores_cos = []
        labels_cos = []

        for i in range(len(outputs_list)):
            for j in range(i + 1, len(outputs_list)):
                id1, tensor1 = outputs_list[i]
                id2, tensor2 = outputs_list[j]

                # Chuyển đổi numpy array về tensor và chuyển lên thiết bị cuda
                tensor1 = torch.tensor(tensor1)
                tensor2 = torch.tensor(tensor2)

                # Tính khoảng cách Euclidean
                score_dis = F.pairwise_distance(tensor1, tensor2).item()
                # Tính cosine similarity
                score_cos = F.cosine_similarity(tensor1.unsqueeze(0), tensor2.unsqueeze(0)).item()

                # Lưu độ tương đồng và nhãn
                scores_dis.append(score_dis)
                labels_dis.append(0 if id1 == id2 else 1)
                scores_cos.append(score_cos)
                labels_cos.append(1 if id1 == id2 else 0)

        y_true_dis = np.array(labels_dis)
        y_scores_dis = np.array(scores_dis)
        fpr_dis, tpr_dis, thresholds_dis = roc_curve(y_true_dis, y_scores_dis)
        roc_auc_dis = auc(fpr_dis, tpr_dis)

        y_true_cos = np.array(labels_cos)
        y_scores_cos = np.array(scores_cos)
        fpr_cos, tpr_cos, thresholds_cos = roc_curve(y_true_cos, y_scores_cos)
        roc_auc_cos = auc(fpr_cos, tpr_cos)
    return roc_auc_dis, roc_auc_cos

In [8]:
margin = 1.
embedding_net = EmbeddingNet_Concat(num_classes = 1024)
model = TripletNet_Concat(embedding_net)


model_path1 = r'/kaggle/input/weight-triplet/weight_2d_triplet.pt'
state_dict1 = torch.load(model_path1, map_location=torch.device('cuda'))
new_state_dict1 = {k[14:]: v for k, v in state_dict1.items()}
model.embedding_net.resnet1.load_state_dict(new_state_dict1, strict=True)

model_path2 = r'/kaggle/input/weight-triplet/weight_3d_triplet.pt'
state_dict2 = torch.load(model_path2, map_location=torch.device('cuda'))
new_state_dict2 = {k[14:]: v for k, v in state_dict1.items()}
model.embedding_net.resnet2.load_state_dict(new_state_dict2, strict=False)

# # Đóng băng resnet1
# for param in model.embedding_net.parameters():
#     param.requires_grad = False

# # Đóng băng resnet2
# for param in model.embedding_net.parameters():
#     param.requires_grad = False

model.eval()

if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-5
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

n_epochs = 1000
log_interval = 1
ts = time.time()
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, save_best_loss=True, path_save='/kaggle/working/best_model.pt', num_epoch_roc=10, train_loader_image=train_loader_image, val_loader_image=val_loader_image)
print(time.time()-ts)
torch.save(model.state_dict(), '/kaggle/working/last_model.pt')

Epoch: 1/1000. Train set: Average loss: 3.3717
Epoch: 1/1000. Validation set: Average loss: 0.8533
Epoch: 2/1000. Train set: Average loss: 3.3335
Epoch: 2/1000. Validation set: Average loss: 0.9889
Epoch: 3/1000. Train set: Average loss: 2.4608
Epoch: 3/1000. Validation set: Average loss: 1.1645
Epoch: 4/1000. Train set: Average loss: 2.7507
Epoch: 4/1000. Validation set: Average loss: 1.6236
Epoch: 5/1000. Train set: Average loss: 1.0880
Epoch: 5/1000. Validation set: Average loss: 0.8934
Epoch: 6/1000. Train set: Average loss: 1.5368
Epoch: 6/1000. Validation set: Average loss: 1.4323
Epoch: 7/1000. Train set: Average loss: 1.1939
Epoch: 7/1000. Validation set: Average loss: 0.9969
Epoch: 8/1000. Train set: Average loss: 2.3209
Epoch: 8/1000. Validation set: Average loss: 0.8730
Epoch: 9/1000. Train set: Average loss: 1.1557
Epoch: 9/1000. Validation set: Average loss: 1.5109
---Epoch: 9/1000. Train set: Distance ROC: 0.9776	Cosine similarity ROC: 0.9736
---Epoch: 9/1000. Validation 